# SKEKB Beam Beam Test

## Python Setup

In [ ]:
import sys

import xobjects as xo
import xtrack as xt
import xfields as xf
import xpart as xp

from xtrack.slicing import Teapot, Strategy

from cpymad.madx import Madx

import numpy as np
import scipy.constants as cst
import matplotlib.pyplot as plt

## User Variables

In [ ]:
########################################
# Simulation Settings
########################################
n_turns             = int(1E2)
n_macroparticles    = int(1E3)
n_slices            = int(101)

########################################
# File Paths
########################################
her_path            = "/Users/jack/Desktop/*Thesis/Analysis/superkekb/no_sol/sher_5781_60_1_nosol_simple.seq"

########################################
# Computation Mode
########################################
context = xo.ContextCpu(omp_num_threads='auto')

## Machine Parameters

### Pre LS1 Parameters

In [ ]:
########################################
# HER Parameters
########################################
her_params = {
    'energy':           7.007,
    'p0c':              7.007E9,
    'bunch_intensity':  3.07E10,
    'n_bunches':        2249,
    'beta_x':           60E-3,
    'beta_y':           1.0E-3,
    'physemit_x':       4.59E-9,
    'physemit_y':       4.62E-11,
    'lattemit_y':       1.00E-11, #Guess
    'Qx':               45.532,
    'Qy':               43.573,
    'sigma_delta':      1.00E-3,
    'sigma_z':          5.10E-3,
    'phi':              41.5E-3,
    'circumference':    3016.315,
}

########################################
# LER Parameters
########################################
ler_params = {
    'energy':           4.00,
    'p0c':              4.0E9,
    'bunch_intensity':  3.69E10,
    'n_bunches':        2249,
    'beta_x':           80E-3,
    'beta_y':           1.0E-3,
    'physemit_x':       4.01E-9,
    'physemit_y':       4.62E-11,
    'lattemit_y':       1.00E-11, #Guess
    'Qx':               44.525,
    'Qy':               46.589,
    'sigma_delta':      1.00E-3,
    'sigma_z':          4.60E-3,
    'phi':              -41.5E-3,
    'circumference':    3016.315,
}

########################################
# Constants
########################################
mass0               = xp.ELECTRON_MASS_EV

## Load HER

In [ ]:
mad = Madx(
    stdout=sys.stdout # Needs to be set to sys.stdout or breaks
)
print("MADX Spawned")
mad.input(
f"""
    SET, FORMAT="19.15f";
    !option, echo;
    option, update_from_parent=true; // new option in mad-x as of 2/2019

    BEAM, PARTICLE=POSITRON, ENERGY={her_params['energy']};

    CALL, FILE="{her_path}";

    USE, SEQUENCE=ASCE;
    
    SXT_ON = 1;
    RF_ON =1;
        
    USE,SEQUENCE=ASCE;
        TWISS;

"""
)
her = xt.Line.from_madx_sequence(
        mad.sequence['ASCE'],
        allow_thick=True,
        deferred_expressions=True,
    )
print("Line Built from MADX Sequence")

## Slicing

In [ ]:
# define strategy for elements and perform slicing
slicing_strategies = [
    Strategy(slicing=Teapot(1)),  # Default catch-all as in MAD-X
    Strategy(slicing=Teapot(4), element_type=xt.Bend),
    Strategy(slicing=Teapot(5), element_type=xt.Quadrupole),
    Strategy(slicing=Teapot(4), element_type=xt.Sextupole),
]

her.discard_tracker()
print("Slicing thick elements...")
her.slice_thick_elements(slicing_strategies)

## Build Reference Particles

In [ ]:
her_ref_particle = xp.Particles(
    mass0   = xp.ELECTRON_MASS_EV,
    q0      = -1,
    p0c     = her_params['p0c']
    )
her.particle_ref = her_ref_particle

her.build_tracker()
her.config.XTRACK_USE_EXACT_DRIFTS = True

## Correct RF

In [ ]:
her_table = her.get_table()
tt_cav = her_table.rows[her_table.element_type=='Cavity']

for nn in tt_cav.name:
    her.element_refs[nn].lag = 180

## Twiss in XSuite

In [ ]:
her_tw_4d = her.twiss(eneloss_and_damping=False, method="4d")
print(f"HER: {(her_tw_4d.qx, her_tw_4d.qy)}")

## Configure Radiation

In [ ]:
her.configure_radiation(model='mean')
her.compensate_radiation_energy_loss()

## Twiss

In [ ]:
her_tw_rad = her.twiss(eneloss_and_damping=True, method="6d")
print(f"HER: {(her_tw_rad['eq_gemitt_x'], her_tw_rad['eq_gemitt_y'])}")

In [ ]:
print(her_tw_rad['bets0'])
print(her_tw_rad['eq_gemitt_x'])
print(her_tw_rad['eq_gemitt_y'])
print(her_tw_rad['eq_gemitt_zeta'])
print(her_tw_rad['eq_nemitt_x'])
print(her_tw_rad['eq_nemitt_y'])
print(her_tw_rad['eq_nemitt_zeta'])

print(np.sqrt(her_tw_rad['bets0'] * her_tw_rad['eq_gemitt_zeta']) * 1000)

In [ ]:
print((her_params['beta_x'], her_params['beta_y']))
print((min(her_tw_rad['betx']), min(her_tw_rad['bety'])))

fig, ax1 = plt.subplots()

ax1.set_xlabel('Component', c='b')
ax1.set_ylabel('Beta_x', c='b')
ax1.plot(her_tw_rad['betx'], c='b')
ax1.tick_params(axis='y', color='b')

ax2 = ax1.twinx()
ax2.set_ylabel('Beta_y', c='orange')
ax2.plot(her_tw_rad['bety'], c='orange')
ax2.tick_params(axis='y', color='orange')

plt.show()

fig, ax1 = plt.subplots()

ax1.set_xlabel('Component', c='b')
ax1.set_ylabel('Dx', c='b')
ax1.plot(her_tw_rad['dx'], c='b')
ax1.tick_params(axis='y', color='b')

ax2 = ax1.twinx()
ax2.set_ylabel('Dy', c='orange')
ax2.plot(her_tw_rad['dy'], c='orange')
ax2.tick_params(axis='y', color='orange')

plt.show()

## Generate Particles

In [ ]:
electrons = xp.generate_matched_gaussian_bunch(
    num_particles               = n_macroparticles,
    total_intensity_particles   = her_params['bunch_intensity'],
    nemitt_x                    = her_tw_rad['eq_nemitt_x'],
    nemitt_y                    = her_tw_rad['eq_nemitt_y'],
    sigma_z                     = np.sqrt(her_tw_rad['bets0'] * her_tw_rad['eq_gemitt_zeta']),
    line                        = her
)   
electrons.name = "electrons"
electrons._init_random_number_generator() # pylint:disable=protected-access

## Build Trackers

In [ ]:
her.discard_tracker()
her.build_tracker(
    _context                = context,
    use_prebuilt_kernels    = False
)

## Configure Radiation

In [ ]:
her.configure_radiation(
    model               = 'quantum',
    model_beamstrahlung = 'quantum'
)

## Create Tracking Records

In [ ]:
record_electrons = {
    "turn": [],
    "alive": [],
    "luminosity": [],
    "photon_power": [],
    "x_av": [],
    "px_av": [],
    "y_av": [],
    "py_av": [],
    "z_av": [],
    "delta_av": [],
    "x_std": [],
    "px_std": [],
    "y_std": [],
    "py_std": [],
    "z_std": [],
    "delta_std": [],
    'emit_x':[],
    'emit_y':[],
    'emit_z':[],
    'u_bs':[]
}

## Scaling Functions

In [ ]:
def power_scale(energy, params):
    power = (
        energy *
        cst.e * # Convert to J
        1E-3 * # Convert to kJ
        params['n_bunches'] * # Bunches per train
        ( cst.c / params['circumference'] ) * # Crossings per second
        ( params['bunch_intensity'] / n_macroparticles )
    )
    return power

def lumi_scale(crossing_luminosity, params):
    luminosity = (
        crossing_luminosity * 
        1E-4 * #m^-2 -> cm^-2
        params['n_bunches'] * # Bunches per train
        ( cst.c / params['circumference'] ) * #times each bunch passes the IP in 1s
        1E-34
    )
    return luminosity

## Initial Values

In [ ]:
def number_alive(particles):
    return np.sum(particles.state[particles.state==1])

def averages(particles):
    x_av     = np.mean(particles.x[particles.state==1]    , axis=0)
    px_av    = np.mean(particles.px[particles.state==1]   , axis=0)
    y_av     = np.mean(particles.y[particles.state==1]    , axis=0)
    py_av    = np.mean(particles.py[particles.state==1]   , axis=0)
    z_av     = np.mean(particles.zeta[particles.state==1] , axis=0)
    delta_av = np.mean(particles.delta[particles.state==1], axis=0)
    return (x_av, px_av, y_av, py_av, z_av, delta_av)

def standard_deviations(particles):
    x_std     = np.std(particles.x[particles.state==1]    , axis=0)
    px_std    = np.std(particles.px[particles.state==1]   , axis=0)
    y_std     = np.std(particles.y[particles.state==1]    , axis=0)
    py_std    = np.std(particles.py[particles.state==1]   , axis=0)
    z_std     = np.std(particles.zeta[particles.state==1] , axis=0)
    delta_std = np.std(particles.delta[particles.state==1], axis=0)
    return (x_std, px_std, y_std, py_std, z_std, delta_std)

def emittances(particles):
    emit_x = np.sqrt(
        np.mean(( particles.x[particles.state==1] -  np.mean(particles.x[particles.state==1], axis=0))**2, axis=0) *\
        np.mean((particles.px[particles.state==1] - np.mean(particles.px[particles.state==1], axis=0))**2, axis=0) -\
        np.mean(( particles.x[particles.state==1] -  np.mean(particles.x[particles.state==1], axis=0)) *\
        (particles.px[particles.state==1] - np.mean(particles.px[particles.state==1], axis=0)), axis=0)**2
    )
    emit_y = np.sqrt(
        np.mean(( particles.y[particles.state==1] -  np.mean(particles.y[particles.state==1], axis=0))**2, axis=0) *\
        np.mean((particles.py[particles.state==1] - np.mean(particles.py[particles.state==1], axis=0))**2, axis=0) -\
        np.mean(( particles.y[particles.state==1] -  np.mean(particles.y[particles.state==1], axis=0)) *\
        (particles.py[particles.state==1] - np.mean(particles.py[particles.state==1], axis=0)), axis=0)**2
    )
    emit_z = np.sqrt(
        np.mean(( particles.zeta[particles.state==1] -  np.mean(particles.zeta[particles.state==1], axis=0))**2, axis=0) *\
        np.mean((particles.delta[particles.state==1] - np.mean(particles.delta[particles.state==1], axis=0))**2, axis=0) -\
        np.mean(( particles.zeta[particles.state==1] -  np.mean(particles.zeta[particles.state==1], axis=0)) *\
        (particles.delta[particles.state==1] - np.mean(particles.delta[particles.state==1], axis=0)), axis=0)**2
    )
    return (emit_x, emit_y, emit_z)

def append_to_records(record, turn, alive, average, stds, emit, luminosity, photon_power, bs_per_turn):
    record["turn"].append(turn)
    record["alive"].append(alive)
    record["x_av"].append(average[0])
    record["px_av"].append(average[1])
    record["y_av"].append(average[2])
    record["py_av"].append(average[3])
    record["z_av"].append(average[4])
    record["delta_av"].append(average[5])
    record["x_std"].append(stds[0])
    record["px_std"].append(stds[1])
    record["y_std"].append(stds[2])
    record["py_std"].append(stds[3])
    record["z_std"].append(stds[4])
    record["delta_std"].append(stds[5])
    record["emit_x"].append(emit[0])
    record["emit_y"].append(emit[1])
    record["emit_z"].append(emit[2])
    record["luminosity"].append(luminosity)
    record["photon_power"].append(photon_power)
    record["u_bs"].append(bs_per_turn)


In [ ]:
append_to_records(
    record          = record_electrons,
    turn            = 0,
    alive           = number_alive(electrons),
    average         = averages(electrons),
    stds            = standard_deviations(electrons),
    emit            = emittances(electrons),
    luminosity      = 0,
    photon_power    = 0,
    bs_per_turn     = 0
)

## Track

In [ ]:
for turn in range(n_turns):
    print(f"Turn: {turn+1}")

    # track for 1 period
    her.track(
        particles=electrons,
        num_turns=1,
        turn_by_turn_monitor=True
    )

    append_to_records(
        record          = record_electrons,
        turn            = turn + 1,
        alive           = number_alive(electrons),
        average         = averages(electrons),
        stds            = standard_deviations(electrons),
        emit            = emittances(electrons),
        luminosity      = lumi_scale(0, params=her_params),
        photon_power    = power_scale(0, params=her_params),
        bs_per_turn     = 0
    )

    # Delete tracker records before next period
    her.tracker._init_io_buffer()

## Plotting

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 4), layout='constrained', sharex=True, sharey=False)

sigma_x_her     = np.sqrt(her_params['beta_x'] * her_params['physemit_x'])
sigma_px_her    = np.sqrt(her_params['physemit_x'] / her_params['beta_x'])

axs[0].plot(record_electrons["turn"], record_electrons["x_av"], label="positrons", c='r')
axs[0].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[0].axhline(y = sigma_x_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].axhline(y = -sigma_x_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].set_ylabel('x')

axs[1].plot(record_electrons["turn"], record_electrons["px_av"], label="positrons", c='r')
axs[1].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[1].axhline(y = sigma_px_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].axhline(y = -sigma_px_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].set_ylabel('px')

axs[2].plot(record_electrons["turn"], record_electrons["x_std"], label="positrons", c='r')
axs[2].axhline(y = sigma_x_her, xmin=0, xmax=n_turns+1, c='r')
axs[2].set_ylabel('Sigma x')

axs[3].plot(record_electrons["turn"], record_electrons["px_std"], label="positrons", c='r')
axs[3].axhline(y = sigma_px_her, xmin=0, xmax=n_turns+1, c='r')
axs[3].set_ylabel('Sigma px')

axs[4].plot(record_electrons["turn"], record_electrons["emit_x"], label="positrons", c='r')
axs[4].axhline(y = her_params['physemit_x'], xmin=0, xmax=n_turns+1, c='r')
axs[4].set_ylabel('Emittance x')

fig.legend(
    axs,
    labels=['Electrons', 'Positrons'],
    loc=(0.85,0.00),
    ncols=2
)

fig.supxlabel('Turns')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 4), layout='constrained', sharex=True, sharey=False)

sigma_y_her     = np.sqrt(her_params['beta_y'] * her_params['physemit_y'])
sigma_py_her    = np.sqrt(her_params['physemit_y'] / her_params['beta_y'])

axs[0].plot(record_electrons["turn"], record_electrons["y_av"], label="positrons", c='r')
axs[0].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[0].axhline(y = sigma_y_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].axhline(y = -sigma_y_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].set_ylabel('y')

axs[1].plot(record_electrons["turn"], record_electrons["py_av"], label="positrons", c='r')
axs[1].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[1].axhline(y = sigma_py_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].axhline(y = -sigma_py_her, xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].set_ylabel('py')

axs[2].plot(record_electrons["turn"], record_electrons["y_std"], label="positrons", c='r')
axs[2].axhline(y = sigma_y_her, xmin=0, xmax=n_turns+1, c='r')
axs[2].set_ylabel('Sigma y')

axs[3].plot(record_electrons["turn"], record_electrons["py_std"], label="positrons", c='r')
axs[3].axhline(y = sigma_py_her, xmin=0, xmax=n_turns+1, c='r')
axs[3].set_ylabel('Sigma py')

axs[4].plot(record_electrons["turn"], record_electrons["emit_y"], label="positrons", c='r')
axs[4].set_ylabel('Emittance y')

fig.legend(
    axs,
    labels=['Electrons', 'Positrons'],
    loc=(0.85,0.0),
    ncols=2
)

fig.supxlabel('Turns')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(16, 4), layout='constrained', sharex=True, sharey=False)

axs[0].plot(record_electrons["turn"], record_electrons["z_av"], label="positrons", c='r')
axs[0].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[0].axhline(y = her_params['sigma_z'], xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].axhline(y = -her_params['sigma_z'], xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[0].set_ylabel('z')

axs[1].plot(record_electrons["turn"], record_electrons["delta_av"], label="positrons", c='r')
axs[1].axhline(y=0, xmin=0, xmax=n_turns+1)
axs[1].axhline(y = her_params['sigma_z'], xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].axhline(y = -her_params['sigma_z'], xmin=0, xmax=n_turns+1, linestyle='--', c='r')
axs[1].set_ylabel('pz')

axs[2].plot(record_electrons["turn"], record_electrons["z_std"], label="positrons", c='r')
axs[2].axhline(y = her_params['sigma_z'], xmin=0, xmax=n_turns+1, c='r')
axs[2].set_ylabel('Sigma z')

axs[3].plot(record_electrons["turn"], record_electrons["delta_std"], label="positrons", c='r')
axs[3].set_ylabel('Sigma pz')

axs[4].plot(record_electrons["turn"], record_electrons["emit_z"], label="positrons", c='r')
axs[4].set_ylabel('Emittance z')

fig.legend(
    axs,
    labels=['Electrons', 'Positrons'],
    loc=(0.85,0.0),
    ncols=2
)

fig.supxlabel('Turns')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4), layout='constrained', sharex=True, sharey=False)
axs[0].plot(record_electrons["turn"], record_electrons["luminosity"], label="positrons", c='r')
axs[0].set_ylabel('Luminosity [$10^34 cm^-2 s^-1$]')

axs[1].plot(record_electrons["turn"], record_electrons["photon_power"], label="positrons", c='r')
axs[1].set_ylabel('Photon Power')

axs[2].plot(record_electrons["turn"], record_electrons["alive"], label="positrons", c='r')
axs[2].set_ylabel('Alive')

fig.legend(
    axs,
    labels=['Electrons', 'Positrons'],
    loc=(0.85,0.0),
    ncols=2
)

fig.supxlabel('Turns')
plt.show()